In [1]:
import numpy as np
import pandas as pd

In [3]:
pred_seqs = pd.read_csv('pred_seqs.csv')
class_labels = pd.read_csv('class_labels.csv')

In [4]:
pred_seqs

,Prediction Sequences
0,[5330000. 220000. 240000.]
1,[5880000. 2180000. 1560000.]
2,[5.0900e+06 4.0000e+05 1.2000e+05 3.4783e+08 4...
3,[4960000. 40000. 30000.]
4,[1.4896e+08 1.2000e+05 1.4000e+05]
...,...
1701225,[4950000. 110000. 100000.]
1701226,[]
1701227,[4.8900000e+06 1.2000000e+05 1.7000000e+05 5.2...
1701228,[]


In [5]:
combined = pd.concat([pred_seqs, class_labels], axis = 1)

In [6]:
combined

,Prediction Sequences,Labels
0,[5330000. 220000. 240000.],0
1,[5880000. 2180000. 1560000.],0
2,[5.0900e+06 4.0000e+05 1.2000e+05 3.4783e+08 4...,0
3,[4960000. 40000. 30000.],0
4,[1.4896e+08 1.2000e+05 1.4000e+05],0
...,...,...
1701225,[4950000. 110000. 100000.],0
1701226,[],0
1701227,[4.8900000e+06 1.2000000e+05 1.7000000e+05 5.2...,1
1701228,[],0


In [7]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

2023-02-03 14:01:36.941374: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/erictria/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# Parameter split_percent defines the ratio of training examples
def get_train_test(url, split_percent=0.8):
    df = read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [10]:
from tensorflow.keras.datasets import imdb

In [12]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

17464789/17464789 [==============================] - 1s 0us/step


In [13]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [16]:
import tensorflow as tf
tf.random.set_seed(7)

In [24]:
#pred_seqs_secs = pred_seqs / 1000000
pred_seqs['Prediction Sequences']

0                               [5330000.  220000.  240000.]
1                               [5880000. 2180000. 1560000.]
2          [5.0900e+06 4.0000e+05 1.2000e+05 3.4783e+08 4...
3                               [4960000.   40000.   30000.]
4                         [1.4896e+08 1.2000e+05 1.4000e+05]
                                 ...                        
1701225                         [4950000.  110000.  100000.]
1701226                                                   []
1701227    [4.8900000e+06 1.2000000e+05 1.7000000e+05 5.2...
1701228                                                   []
1701229                                                   []
Name: Prediction Sequences, Length: 1701230, dtype: object

In [45]:
pred_seqs['Prediction Sequences'][1701227]

'[4.8900000e+06 1.2000000e+05 1.7000000e+05 5.2952378e+11]'

In [26]:
test = pred_seqs['Prediction Sequences'][2]
test

'[5.0900e+06 4.0000e+05 1.2000e+05 3.4783e+08 4.9700e+06 1.6590e+07\n 2.8392e+08]'

In [67]:
def clean_str(my_str):
    test_clean = my_str.replace('[', '').replace(']', '')
    test_arr = test_clean.split(' ')
    test_arr = list(filter(lambda x: x != '' and x != '...', test_arr))
    if len(test_arr) == 0:
        return np.array(test_arr)
    try:
        final_arr = [float(t) for t in test_arr]
    except Exception as e:
        print(e)
        print(test_arr)
        return
    return np.array(final_arr)

In [68]:
pred_seqs_2 = pred_seqs['Prediction Sequences'].apply(clean_str)

In [69]:
pred_seqs_2

0                            [5330000.0, 220000.0, 240000.0]
1                          [5880000.0, 2180000.0, 1560000.0]
2          [5090000.0, 400000.0, 120000.0, 347830000.0, 4...
3                              [4960000.0, 40000.0, 30000.0]
4                          [148960000.0, 120000.0, 140000.0]
                                 ...                        
1701225                      [4950000.0, 110000.0, 100000.0]
1701226                                                   []
1701227      [4890000.0, 120000.0, 170000.0, 529523780000.0]
1701228                                                   []
1701229                                                   []
Name: Prediction Sequences, Length: 1701230, dtype: object

In [77]:
pred_time_seqs_arr = np.array(pred_seqs_2, dtype=object)
class_labels_arr = np.array(class_labels, dtype=object)
class_labels_arr = np.asarray(class_labels_arr).astype('float32')

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pred_time_seqs_arr, class_labels_arr, test_size=0.2, random_state=6013)



In [79]:
X_train = sequence.pad_sequences(X_train, maxlen=24, value=-1.)
X_test = sequence.pad_sequences(X_test, maxlen=24, value=-1.)

In [84]:
from tensorflow.keras.layers import Dropout

In [86]:
# create the model
#embedding_vecor_length = 32
#max_sequence_length = 24
#top_words = 5000
model = Sequential()
#model.add(Embedding(top_words, embedding_vecor_length, input_length=max_sequence_length))
#model.add(LSTM(100))
#model.add(Dense(1, activation='sigmoid'))

model.add(LSTM(50, input_shape = (24, 1)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 50)                10400     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_14 (Dense)            (None, 10)                510       
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 10,921
Trainable params: 10,921
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
21266/21266 [====================